In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

import sys, os
sys.path.append('/'.join(os.getcwd().split('/')[:4]))
from config.get import cfg

2022-01-11 14:03:46.449535: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.2


In [2]:
import pandas_ta as ta

In [3]:
data = pd.read_csv(cfg['files']['liquid']['preprocessed_data'],nrows=10_000_000).drop(columns=['Unnamed: 0', 'baseAmount','quoteAmount', 'gasValue','time'])

In [4]:
data.head()

,cycle_id,token1,token2,quotePrice,gasPrice
0,1002,WETH,BUIDL,22550.438239,20.000000
1,1002,WETH,BUIDL,9462.858024,101.000004
2,1002,WETH,BUIDL,32966.161101,30.000001
3,1002,WETH,BUIDL,20101.046750,31.000001
4,1002,WETH,BUIDL,14104.537180,31.000300


# Rule based indicators

> **Note**: we tried to use the ta.Strategy but it was too slow on a Grouper object

The following indicators are builts using two windows(5 and 20) on both prices and log returns:

1. SMA
5. volatily
  


In [5]:
def logret(data, cols = ['quotePrice', 'gasPrice']):
    """Constructs log return series for each col in cols per token pair data"""
    fcols = []
    for col in cols:
        ncol = f'logret_{col}'
        data[ncol] = np.log(data[col])
        fcols.append(ncol)
    
    grouped = data.groupby(['cycle_id', 'token1','token2'])[fcols]
    log_ret = grouped.diff()
    data.drop(columns=fcols,inplace=True)
    return log_ret

In [6]:
logret_data = logret(data)

/ssoft/spack/arvine/v1/opt/spack/linux-rhel7-skylake_avx512/gcc-8.4.0/python-3.7.7-drpdlwdbo3lmtkcbckq227ypnzno4ek3/lib/python3.7/site-packages/pandas/core/series.py:679: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [7]:
for c in logret_data.columns:
    data[c] = logret_data[c]

In [8]:
data.head()

,cycle_id,token1,token2,quotePrice,gasPrice,logret_quotePrice,logret_gasPrice
0,1002,WETH,BUIDL,22550.438239,20.000000,NaN,NaN
1,1002,WETH,BUIDL,9462.858024,101.000004,-0.868380,1.619388
2,1002,WETH,BUIDL,32966.161101,30.000001,1.248107,-1.213923
3,1002,WETH,BUIDL,20101.046750,31.000001,-0.494710,0.032790
4,1002,WETH,BUIDL,14104.537180,31.000300,-0.354275,0.000010


In [34]:
def get_ta_data(data, windows):
    dfs = []
    grouped = data.groupby(['cycle_id', 'token1','token2'])
    for window in windows:
        print("////////////")
        g = grouped.rolling(window).agg({
                 'quotePrice'        : ['mean','std'],
                 'gasPrice'          : ['mean','std'],
                 'logret_quotePrice' : ['mean','std'],
                 'logret_gasPrice'   : ['mean','std'], 
             })
        new_cols = [f'quotePrice_{window}'       ,f'gasPrice_{window}',
                    f'logret_quotePrice_{window}',f'logret_gasPrice_{window}' ]
        g.columns.set_levels(new_cols,level=0,inplace=True)
        dfs.append(g)
    return pd.concat(dfs, axis=1)

In [35]:
get_ta_data(pd.concat((data, logret_data),axis=1),[5,20])

////////////


TypeError: aggregate() missing 1 required positional argument: 'func'

In [44]:
window = 5
def get_ta_data(window):
    g = data.groupby(['cycle_id', 'token1','token2']).rolling(window).agg({
                     'quotePrice'        : ['mean','std'],
                     'gasPrice'          : ['mean','std'],
                     'logret_quotePrice' : ['mean','std'],
                     'logret_gasPrice'   : ['mean','std'], 
            })
    new_cols = [f'quotePrice_{window}'       ,f'gasPrice_{window}',
                f'logret_quotePrice_{window}',f'logret_gasPrice_{window}' ]
    g.columns.set_levels(new_cols,level=0,inplace=True)
    return g.fillna(0.0)

In [ ]:
ta_data_20 = get_ta_data(20)

In [39]:
ta_data_5 = g

In [41]:
ta_data_5.fillna(0.0)

quotePrice_5            gasPrice_5             \
                                       mean       std        mean        std   
cycle_id token1 token2                                                         
1002     BUIDL  STA    600         0.000000  0.000000    0.000000   0.000000   
                       601         0.000000  0.000000    0.000000   0.000000   
                       602         0.000000  0.000000    0.000000   0.000000   
                       603         0.000000  0.000000    0.000000   0.000000   
                       604         8.742736  3.170428   46.423853  13.554863   
...                                     ...       ...         ...        ...   
47099    WETH   SUSHI  9998906    93.903858  0.820166  195.599997  75.427452   
                       9998907    93.590110  0.935031  170.599996  26.959229   
                       9998908    93.576702  0.916728  167.099997  30.406413   
                       9998909    93.845372  1.368933  154.403997  26.196298   
                       9998910    93.910761  1.331022  152.204000  27.104759   

                               logret_quotePrice_5            \
                                              mean       std   
cycle_id token1 token2                                         
1002     BUIDL  STA    600                0.000000  0.000000   
                       601                0.000000  0.000000   
                       602                0.000000  0.000000   
                       603                0.000000  0.000000   
                       604                0.000000  0.000000   
...                                            ...       ...   
47099    WETH   SUSHI  9998906           -0.000384  0.010213   
                       9998907           -0.003356  0.007403   
                       9998908           -0.000142  0.012788   
                       9998909            0.002821  0.014302   
                       9998910            0.000701  0.018058   

                               logret_gasPrice_5            
                                            mean       std  
cycle_id token1 token2                                      
1002     BUIDL  STA    600              0.000000  0.000000  
                       601              0.000000  0.000000  
                       602              0.000000  0.000000  
                       603              0.000000  0.000000  
                       604              0.000000  0.000000  
...                                          ...       ...  
47099    WETH   SUSHI  9998906          0.018387  0.605447  
                       9998907         -0.097102  0.430007  
                       9998908         -0.023960  0.306635  
                       9998909         -0.076369  0.266876  
                       9998910         -0.015232  0.240029  

[10000000 rows x 8 columns]

In [ ]:
def build_ta(np_array):
    df = pd.DataFrame(data=np.array, columns=['quotePrice','gasPrice'])
    df.rolling(window).agg({
                 'quotePrice'        : ['mean','std'],
                 'gasPrice'          : ['mean','std'],
                 'logret_quotePrice' : ['mean','std'],
                 'logret_gasPrice'   : ['mean','std'], 
             })

# From numpy

In [14]:
X_train = np.load(cfg['files']['liquid']["raw_train_features"])
X_test  = np.load(cfg['files']['liquid']["raw_test_features"])

In [16]:
N, T, P, K = X_train.shape

In [19]:
tX_train = np.zeros((N, T, P, K * 2 * 2))
for n in range(N):
    for t in range(T):
        np_array = X_train[n,t,:,:]
        break
    break
        #tX_train[n,t,:,:] 

In [20]:
np_array.shape

(600, 2)

In [ ]:
df = pd.DataFrame(data=np_array, columns=['quotePrice','gasPrice'])
def logret(data):
    print(np.diff(np.log(data.quotePrice)).shape)
    data['logret_quotePrice'] = np.diff(np.log(data.quotePrice))
    data['logret_gasPrice']   = np.diff(np.log(data.gasPrice))
logret(df)
df.rolling(2).agg({
                 'quotePrice'        : ['mean','std'],
                 'gasPrice'          : ['mean','std'],
                 'logret_quotePrice' : ['mean','std'],
                 'logret_gasPrice'   : ['mean','std'], 
             })